In [2]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from parse import *
import seaborn as sns

In [3]:
CHUNKSIZE = 100000

TRAIN_PATH        = "./train_StationPathInfo.csv"
TRAIN_PATH_EX     = "./train_StationPathInfoEx.csv"
TEST_PATH         = "./test_StationPathInfo.csv"
TEST_PATH_EX      = "./test_StationPathInfoEx.csv"
TRAIN_NUMERIC     = "../"
TEST_NUMERIC      = "../"

SEED = 0
CHUNKSIZE = 50000
NROWS = 1200000

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
    
    
    

In [4]:
# 読み出す。

df_train_bin = pd.read_csv(TRAIN_PATH)
df_test_bin  = pd.read_csv(TEST_PATH)
df_pass = pd.concat([df_train_bin, df_test_bin])


通過ステーションをヒートマップにしてみる。

In [5]:
# 時系列でソートしたヒートマップを出してみる。

TEST_START_TIME      = "./test_StartEndTime.csv"
TRAIN_START_TIME     = "./train_StartEndTime.csv"

df_start_train = pd.read_csv(TRAIN_START_TIME)
df_start_train = df_start_train.ix[:,['Id','StartTime','EndTime', 'Response']]
df_start_test = pd.read_csv(TEST_START_TIME)
df_start_test['Response'] = -1


C:\Users\Tomonobu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


FileNotFoundError: File b'./test_StartEndTime.csv' does not exist

In [ ]:
df_start_train.head()

In [ ]:
df_start_test.head()
df_start = pd.concat([df_start_train, df_start_test])
df_start.head()

In [ ]:
df_train = pd.merge(df_start, df_pass, how='left')
df_train = df_train.fillna(-1500)




In [ ]:
df_train[df_train.isnull().any(axis=1)]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


def separate_X_y(df):
    X = df.drop(['Response'], axis=1)
    y = df['Response']
    return X, y

def train_with_r_forest(df):
    X_train, y_train = separate_X_y(df)

    cl_weight = {0:0.3, 1:0.7}
    
    rf = RandomForestClassifier(max_depth=5,n_estimators=10)
    rf.fit(X_train, y_train)
    return rf


In [ ]:
def train_and_predict_one_chnuk(df_train):
    df_train_ok   = df_train[df_train['Response'] == 0]
    df_train_ng   = df_train[df_train['Response'] == 1]
    df_test = df_train[df_train['Response'] == -1]
    
    undersample_rate = len(df_train_ng) * 3 / len(df_train_ok)
    df_train_ok_sample = df_train_ok.sample(frac = undersample_rate)

    df_train_balance = pd.concat([df_train_ok_sample, df_train_ng])
    
    rf = train_with_r_forest(df_train_balance)

    X_test, y_test = separate_X_y(df_test)
    y_pred = rf.predict(X_test)
      
    df_result_add = pd.DataFrame(columns=['Id', 'Response'])
    df_result_add.loc[:, 'Id']       = df_test['Id'].values
    df_result_add.loc[:, 'Response'] = y_pred
    
    return df_result_add

df_result = train_and_predict_one_chnuk(df_train)
    

In [ ]:
df_result[df_result['Response']==1].shape


In [ ]:
df_result[df_result['Response']==0].shape

In [ ]:
df_result.to_csv("submit_20180619_2.csv", index=False)